# Chat Prompt

## Load Database

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.modelscope_hub import ModelScopeEmbeddings
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
persist_directory = "/workdir/data_base/vector_db"

embd_model_dir = "/workdir/data_base/llm_models/ModelScope/iic/nlp_gte_sentence-embedding_chinese-large"
ms_gte_embedding = ModelScopeEmbeddings(
    model_id=embd_model_dir, # 
    model_revision="v1.1.0",
)

vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=ms_gte_embedding
)

## Load LLM

In [ ]:
import sys
sys.path.append("../llm")
from ChatGLM3 import ChatGLM3

llm = ChatGLM3()
llm.load_model(
    "/workdir/data_base/llm_models/ModelScope/ZhipuAI/chatglm3-6b"
)

## Build Prompt

In [3]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context","question"],
    template=template
)

In [4]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt":QA_CHAIN_PROMPT}
)

## Test Prompt

In [ ]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

result = qa_chain({"query": question_1})
print(result["result"])

In [ ]:
result = qa_chain({"query": question_2})
print(result["result"])

In [ ]:
prompt_template = """请回答下列问题：
    {}""".format(question_2)
llm.predict(prompt_template)

# Conversation History

## Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## ConversationalRetrievalChain

1. 将之前的对话与新问题合并生成一个完整查询语句；
2. 在向量数据库中搜索该查询的相关文档；
3. 获取结果后，存储所有答案到对话记忆区；
4. 用户可在UI中查看完整对话流程。

In [ ]:
# Load vector db from block no.1
# Load llm from block no.2

from langchain.chains import ConversationalRetrievalChain

conv_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(),
    memory=memory
)

